## Load Modules

In [ ]:
#!pip install pysal -q

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import mapclassify as mc
import pandas as pd
import numpy as np
import seaborn as sns
import contextily as ctx
from shapely.geometry import Polygon
from pysal.lib import cg as geometry
from pysal.lib import weights

## Data Exploration

In [ ]:
scot = gpd.read_file("scot_obesity.gpkg")
scot = scot.set_index('GSS_CODEWD', drop=False)

In [ ]:
scot.head(2)

In [ ]:
scot.explore()

In [ ]:
wd_names = sorted(scot['WD_Name'].unique())

In [ ]:
scot.describe()


In [ ]:
scot.shape

## Choropleth Maps and summary statistics

In [ ]:
jenks = mc.NaturalBreaks(scot['Avg_Obesit'], k=5)

In [ ]:
jenks

you may want to change this to `mapclassify.EqualInterval()` or to `mapclassify.Quantiles()`

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
scot.plot(column="jenks_class", 
        cmap='YlGnBu', # Choose a colormap
        ax=ax, 
        legend=True,
        classification_kwds={'bins': jenks.bins})
ax.set_title('Choropleth Map with Jenks Classification')
ax.axis('off')

plt.show()

## Global Moran's I

### Building spatial weight

In [ ]:
# Build Queen contiguity
queen = weights.contiguity.Queen.from_dataframe(scot, use_index=True)
print(queen)

# Build Rook contiguity
rook = weights.contiguity.Rook.from_dataframe(scot, use_index=True)
print(rook)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

scot.plot(ax=axs[0], edgecolor='black', facecolor='lightgrey')
queen.plot(scot, ax=axs[0], edge_kws=dict(color='red', linestyle=":", linewidth=1))
axs[0].set_title("Queen Contiguity")

scot.plot(ax=axs[1], edgecolor='black', facecolor='lightgrey')
rook.plot(scot, ax=axs[1], edge_kws=dict(color='blue', linestyle=":", linewidth=1))
axs[1].set_title("Rook Contiguity")

plt.show()

In [ ]:
print(queen.n) # n = the number of observations (spatial units) in your GeoDataFrame
print(queen.pct_nonzero) # Percentage of non-zero entries in the spatial weight matrix
print(rook.n)
print(rook.pct_nonzero)

In [ ]:
queen.cardinalities

In [ ]:
card_series = pd.Series(queen.cardinalities)
card_series.name = "num_neighbours"
print(card_series.sort_values(ascending=False))
print(card_series.describe())  # mean, std, min, max

sns.displot(card_series, kde=True)
#card_series.hist(bins=10)

In [ ]:
from esda import Moran

In [ ]:
scot = scot.reset_index(drop=True)

In [ ]:
scot.explore(
    "Avg_Obesit",
    cmap="coolwarm",
    vmin=2,
    vmax=17,
    prefer_canvas=True,
    tiles="CartoDB Positron",
)

In [ ]:
y = scot["Avg_Obesit"].values

In [ ]:
# Using Queen contiguity
moran_q = Moran(y, queen)
print(f"Queen Moran's I: {moran_q.I}, p-value: {moran_q.p_sim}")

# Using Rook contiguity
moran_r = Moran(y, rook)
print(f"Rook Moran's I: {moran_r.I}, p-value: {moran_r.p_sim}")

In [ ]:
from splot.esda import moran_scatterplot

In [ ]:
# Create Moran scatterplot
fig, ax = moran_scatterplot(moran_q, aspect_equal=True)

# Adjust plot
ax.set_title("Moran Scatterplot: Average Obesity in Scotland", fontsize=14)
ax.set_xlabel("Average Obesity in Scotland (standardised)", fontsize=12)
ax.set_ylabel("Spatial Lag", fontsize=12)
plt.grid(True)
plt.show()

## Local Moran’s I (LISA)

In [ ]:
from esda import Moran_Local
from splot.esda import lisa_cluster

In [ ]:
lisa = Moran_Local(y, queen)

In [ ]:
lisa

In [ ]:
lisa.plot_scatter()

In [ ]:
scot['cluster'] = lisa.get_cluster_labels(crit_value=0.05)

In [ ]:
# Frequency table (includes 'Not Significant')
freq_table = scot['cluster'].value_counts()

# Display
print(freq_table)

# Total number of electoral wards
total_wards = freq_table.sum()
print(f"Total wards: {total_wards}")

In [ ]:
# Percentages
percentages = (freq_table / total_wards) * 100

freq_table_df = pd.DataFrame({
    'Count': freq_table,
    'Percentage': percentages.round(1)
})

# Display
freq_table_df

## Getis-Ord Gi*

In [ ]:
from esda import G_Local

In [ ]:
y = scot['Avg_Obesit'].astype(float).values
# Calculate Local G (Gi*)
gi = G_Local(y, queen, permutations=999)

In [ ]:
# Attach results
scot['GiZ'] = gi.z_sim
scot['GiP'] = gi.p_sim
scot['Gi_sig'] = scot['GiP'] < 0.05

In [ ]:
# Categorise
def hotspot_coldspot(z):
    if z > 0:
        return 'Hotspot'
    else:
        return 'Coldspot'

scot['Gi_cluster'] = scot.apply(
    lambda row: hotspot_coldspot(row['GiZ']) if row['Gi_sig'] else 'Not Significant',
    axis=1
)

In [ ]:
# Define your custom colors
colour_dict = {
    'Hotspot': '#e31a1c',  # Strong Red
    'Coldspot': '#1f78b4',  # Nice Blue
    'Not Significant': '#d9d9d9'  # Light Grey
}

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

# Use the cluster column for plotting
scot.plot(
    column='Gi_cluster',
    categorical=True,
    legend=True,
    cmap=None,  # Don't use default color maps
    color=scot['Gi_cluster'].map(colour_dict),
    edgecolor='black',
    linewidth=0.5,
    ax=ax
)

# Customise the plot
plt.title('Getis-Ord Gi* Hotspots and Coldspots of Scottish Obesity', fontsize=15)
plt.axis('off')
plt.show()


## Other variables

Test other variables for the Moran's I value, such as nocar, and socrent.